In [1]:
import torch
from torch import tensor
import matplotlib.pyplot as plt

In [52]:
def flattenShape(shape):
	res = 1
	for x in shape:
		res *= x
	return res

class nlayer:
	def __init__(self, shape):
		self.shape = shape
		global layerid
		self.w = {}
		self.trace = {}
		self.nextV = torch.zeros(flattenShape(shape))
	def input(self, otherLayer):
		if not otherLayer in self.w:
			self.w[otherLayer] = torch.rand((flattenShape(otherLayer.shape),flattenShape(self.shape)))*0.2+0.4
		self.nextV += otherLayer.v @ self.w[otherLayer]
	def inputTensor(self, t):
		self.nextV += t.flatten()
	def update(self, reward=0.0, lr=0.5):
		self.v = self.nextV
		self.nextV = torch.zeros(flattenShape(self.shape))
		for otherLayer in self.w:
			if not otherLayer in self.trace:
				self.trace[otherLayer] = torch.zeros((flattenShape(otherLayer.shape),flattenShape(self.shape)))
			self.trace[otherLayer] += otherLayer.v[:,None] * self.v[None,:]
			self.w[otherLayer] += reward*self.trace[otherLayer]*lr
			self.trace[otherLayer] *= max(1.0-abs(reward), 0.0) # replace with learning signal like max(reward,punish)
		return self.v

In [23]:
type(list(layer2.w)[0])

__main__.nlayer

In [27]:
print(dict[str,int]())

dict[str, int]


In [43]:
layer01 = nlayer((3,))
layer02 = nlayer((4,))
layer02.w[layer01] = tensor([[1,0,0,0],[0,0,0,0],[0,0,1,0]],dtype=torch.float)

layer01.inputTensor(tensor([1,1,1]))
layer01.update()
layer02.input(layer01)
out = layer02.update()
print(out)
print(layer02.trace)

tensor([1., 0., 1., 0.])
{<__main__.nlayer object at 0x126f16510>: tensor([[1., 0., 1., 0.],
        [1., 0., 1., 0.],
        [1., 0., 1., 0.]])}


Aug 10, 2025, 3:10 pm: it didn't learn anything even after 50 iterations

In [ ]:
fig, axs = plt.subplots(1, 2)

position = [2,2]

layer1 = nlayer((10,10))
layer2 = nlayer((4,))

reward = 0.0

for i in range(50):
	input = torch.zeros((10,10))
	input[round(position[0]),round(position[1])] = 1.
	layer1.inputTensor(input)
	layer1.update(reward)
	layer2.input(layer1)
	out = layer2.update(reward)

	position[0] += float(out[0]-out[1])
	position[1] += float(out[2]-out[3])

	reward = float(out[0]-out[1])
	# position[0]>8 and position[0]<10 and position[1]>3 and position[1]<7

	print(out,reward)
	axs[0].imshow(input, interpolation='nearest', vmin=0,vmax=1)
	axs[1].imshow(layer2.w[layer1], interpolation='nearest', vmin=0,vmax=1)
	plt.pause(0.1)